In [21]:
# !pip3 install moviepy piexif --break-system-packages

# Libraries

In [22]:
import os
import cv2
import itertools

import numpy as np
import pandas as pd

from collections import Counter
from pprint import pprint
from tqdm import tqdm

from moviepy import VideoFileClip
# from moviepy.editor import VideoFileClip
from PIL import Image
from PIL.TiffImagePlugin import ImageFileDirectory_v2

# User Defined Functions

In [23]:
############################################################
# Get a list of all files and folders in the specified directory
############################################################
def get_folders_in_directory(directory):
    items = os.listdir(directory)
    
    # Filter out the items that are folders
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]    
    return folders

############################################################
# Get a list of all files and folders in the specified directory
############################################################
def list_directory_contents(directory):
    items = os.listdir(directory)    
    return items

############################################################
# Perform image rotation
############################################################
def correct_rotation(frame, rotateCode):
    return cv2.rotate(frame, rotateCode)

############################################################
# Get exactly k frames
############################################################
def check_k_frames(total_frames, n):
    interval = total_frames // n
    extracted_frames = []
    for i in range(n):
        frame_index = i * interval
        extracted_frames.append(frame_index)
    return extracted_frames

############################################################
# Get n frame per second
############################################################
def get_n_frames_per_second(total_frames, fps, n):
    frames_per_second = []
    for second in range(total_frames // fps):
        interval = fps // n
        for i in range(n):
            frame_index = second * fps + i * interval
            if frame_index < total_frames:
                frames_per_second.append(frame_index)
    return frames_per_second


############################################################
# Extract n frames from videos
############################################################
def extract_frames(video_path, output_folder, fps, scale_percent, max_frames):
    # Set an approximate maximum frames allowed
    # Default 30, change to 60 to see if more images helps    
    fps = int(fps)
    
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Check if the video file opened successfully
    if not video.isOpened():
        print("Error: Could not open video file.")
        return
    
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    # list_frames = check_k_frames(total_frames, max_frames)
    list_frames = get_n_frames_per_second(total_frames, fps, max_frames)
    
    frame_count = 0
    saved_frame_count = 0
    
    while True:
        # Read a frame from the video
        ret, frame = video.read()
        
        # If the frame was not read successfully, break the loop
        if not ret:
            break
        
        # Correct the rotation of the frame if needed
        rotateCode = cv2.ROTATE_90_COUNTERCLOCKWISE  # Change this based on the video's rotation metadata
        frame = correct_rotation(frame, rotateCode)
        
        width = int(frame.shape[1] * scale_percent / 100)
        height = int(frame.shape[0] * scale_percent / 100)
        dim = (width, height)
        frame = cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)
        
        # Save every 10th frame as an image file
        if frame_count in list_frames:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_count:04d}.jpeg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_count += 1
        
        frame_count += 1

    # Release the video file
    video.release()
    
    return saved_frame_count


############################################################
# Get video duration
############################################################
def get_video_duration(file_path):
    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = frame_count / fps
    return duration, frame_count, fps


# Load folder

In [24]:
# Specify the directory path
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE'

# Get the list of folders in the directory
folders = get_folders_in_directory(directory_path)
folders = [x for x in folders if 'Subject' in x]

folders.sort()

# Print the list of folders
print(folders)
print(len(folders))

['1. Subject 1', '10. Subject 10', '11. Subject 11', '12. Subject 12', '13. Subject 13', '14. Subject 14', '15. Subject 15', '16. Subject 16', '17. Subject 17', '18. Subject 18', '19. Subject 19', '2. Subject 2', '20. Subject 20', '21. Subject 21', '22. Subject 22', '23. Subject 23', '24. Subject 24', '25. Subject 25', '26. Subject 26', '27. Subject 27', '28. Subject 28', '29. Subject 29', '3. Subject 3', '30. Subject 30', '31. Subject 31', '32. Subject 32', '33. Subject 33', '34. Subject 34', '35. Subject 35', '36. Subject 36', '37. Subject 37', '38. Subject 38', '39. Subject 39', '4. Subject 4', '40. Subject 40', '41. Subject 41', '42. Subject 42', '43. Subject 43', '44. Subject 44', '45. Subject 45', '46. Subject 46', '47. Subject 47', '48. Subject 48', '49. Subject 49', '5. Subject 5', '50. Subject 50', '51. Subject 51', '52. Subject 52', '53. Subject 53', '54. Subject 54', '55. Subject 55', '56. Subject 56', '57. Subject 57', '58. Subject 58', '59. Subject 59', '6. Subject 6', '60

# Get videos counts

In [25]:
list_video_counter = []
for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    
    subject = folder.split('.')[0]
    
    # Get the list of files and folders in the directory
    contents = list_directory_contents(folder_path)
    contents = [x for x in contents if ':Zone.Identifier' not in x]

    print('#'*60)
    print('Subject', subject, ':', contents)
    list_video_counter.append(len(contents))

############################################################
Subject 1 : ['1D0L.mp4', '1D0R.mp4']
############################################################
Subject 10 : ['10D2L.mp4', '10D1R.mp4', '10D1L.mp4', '10D0R.mp4', '10D2R.mp4', '10D0L.mp4']
############################################################
Subject 11 : ['11D0L.mp4', '11D2R.mp4', '11D2L.mp4', '11D1R.mp4', '11D1L.mp4', '11D0R.mp4']
############################################################
Subject 12 : ['12D0L.mp4', '12D0R.mp4']
############################################################
Subject 13 : ['13D1L.mp4', '13D0L.mp4', '13D2L.mp4', '13D5L.mp4']
############################################################
Subject 14 : ['14D1L.mp4', '14D1R.mp4', '14D0R.mp4', '14D0L.mp4']
############################################################
Subject 15 : ['15D2L.mp4', '15D1L.mp4', '15D0R.mp4', '15D0L.mp4', '15D1R.mp4', '15D2R.mp4']
############################################################
Subject 16 : ['16D1R.mp4', 

In [26]:
print('Total Videos:', sum(list_video_counter))
print('Mean Videos :', np.mean(list_video_counter))
print('Max Videos  :', np.max(list_video_counter))
print('Min Videos  :', np.min(list_video_counter))

Total Videos: 425
Mean Videos : 5.985915492957746
Max Videos  : 10
Min Videos  : 2


# Get video duration

In [27]:
list_video_duration = []
list_video_frames = []
list_video_fps = []

for folder in folders:
    folder_path = os.path.join(directory_path, folder)
    
    subject = folder.split('.')[0]
    
    # Get the list of files and folders in the directory
    contents = list_directory_contents(folder_path)
    contents = [x for x in contents if ':Zone.Identifier' not in x]

    for content in contents:
        video_path = os.path.join(folder_path, content)
        duration, frame_count, fps = get_video_duration(video_path)
        list_video_duration.append(duration)
        list_video_frames.append(frame_count)
        list_video_fps.append(fps)
        
        print(f'Subject {subject} - {content} - Duration: {round(duration, 3)}s - {frame_count} frames - FPS {round(fps, 3)}')


Subject 1 - 1D0L.mp4 - Duration: 10.0s - 299 frames - FPS 29.9
Subject 1 - 1D0R.mp4 - Duration: 6.8s - 204 frames - FPS 30.0
Subject 10 - 10D2L.mp4 - Duration: 14.867s - 446 frames - FPS 30.0
Subject 10 - 10D1R.mp4 - Duration: 10.567s - 316 frames - FPS 29.906
Subject 10 - 10D1L.mp4 - Duration: 11.067s - 332 frames - FPS 30.0
Subject 10 - 10D0R.mp4 - Duration: 12.667s - 370 frames - FPS 29.211
Subject 10 - 10D2R.mp4 - Duration: 14.167s - 425 frames - FPS 30.0
Subject 10 - 10D0L.mp4 - Duration: 10.9s - 327 frames - FPS 30.0
Subject 11 - 11D0L.mp4 - Duration: 11.2s - 336 frames - FPS 30.0
Subject 11 - 11D2R.mp4 - Duration: 8.233s - 247 frames - FPS 30.0
Subject 11 - 11D2L.mp4 - Duration: 9.233s - 276 frames - FPS 29.892
Subject 11 - 11D1R.mp4 - Duration: 7.667s - 230 frames - FPS 30.0
Subject 11 - 11D1L.mp4 - Duration: 9.0s - 269 frames - FPS 29.889
Subject 11 - 11D0R.mp4 - Duration: 11.633s - 349 frames - FPS 30.0
Subject 12 - 12D0L.mp4 - Duration: 10.2s - 306 frames - FPS 30.0
Subject 

In [28]:
print('Total Videos:', round(sum(list_video_duration), 3), 's')
print('Mean Videos :', round(np.mean(list_video_duration), 3), 's')
print('Median Videos :', round(np.median(list_video_duration), 3), 's')
print('Max Videos  :', round(np.max(list_video_duration), 3), 's')
print('Min Videos  :', round(np.min(list_video_duration), 3), 's')
print()
print('Total Frames:', round(sum(list_video_frames), 3))
print('Mean Frames :', round(np.mean(list_video_frames), 3))
print('Median Frames :', round(np.median(list_video_frames), 3))
print('Max Frames  :', round(np.max(list_video_frames), 3))
print('Min Frames  :', round(np.min(list_video_frames), 3))

Total Videos: 8237.08 s
Mean Videos : 19.381 s
Median Videos : 20.033 s
Max Videos  : 34.766 s
Min Videos  : 4.633 s

Total Frames: 246972
Mean Frames : 581.111
Median Frames : 600.0
Max Frames  : 1043
Min Frames  : 139


# Extract frames as video to be saved as images

In [29]:
folders

['1. Subject 1',
 '10. Subject 10',
 '11. Subject 11',
 '12. Subject 12',
 '13. Subject 13',
 '14. Subject 14',
 '15. Subject 15',
 '16. Subject 16',
 '17. Subject 17',
 '18. Subject 18',
 '19. Subject 19',
 '2. Subject 2',
 '20. Subject 20',
 '21. Subject 21',
 '22. Subject 22',
 '23. Subject 23',
 '24. Subject 24',
 '25. Subject 25',
 '26. Subject 26',
 '27. Subject 27',
 '28. Subject 28',
 '29. Subject 29',
 '3. Subject 3',
 '30. Subject 30',
 '31. Subject 31',
 '32. Subject 32',
 '33. Subject 33',
 '34. Subject 34',
 '35. Subject 35',
 '36. Subject 36',
 '37. Subject 37',
 '38. Subject 38',
 '39. Subject 39',
 '4. Subject 4',
 '40. Subject 40',
 '41. Subject 41',
 '42. Subject 42',
 '43. Subject 43',
 '44. Subject 44',
 '45. Subject 45',
 '46. Subject 46',
 '47. Subject 47',
 '48. Subject 48',
 '49. Subject 49',
 '5. Subject 5',
 '50. Subject 50',
 '51. Subject 51',
 '52. Subject 52',
 '53. Subject 53',
 '54. Subject 54',
 '55. Subject 55',
 '56. Subject 56',
 '57. Subject 57',
 '5

In [30]:
df_results = []

# folders = ['71. Subject 71',]

list_algorithm = [
    '', 
    # '_colmap', 
    # '_mve', 
    # '_vggsfm'
    ]

# list_scale_percent = [100, 50]
# list_max_frames = [30, 45, 60, 75]

# list_scale_percent = [50]
# list_scale_percent = [25]
# list_scale_percent = [50]
# list_max_frames = [60] # Total no of frame
# list_max_frames = [5] # No of frame per second

list_scale_percent = [100]
list_max_frames = [5] # No of frame per second

for scale_percent, max_frames in itertools.product(list_scale_percent, list_max_frames):
    for folder in folders:
        folder_path = os.path.join(directory_path, folder)
        
        subject = folder.split('.')[0]
        
        # Get the list of files and folders in the directory
        contents = list_directory_contents(folder_path)
        contents = [x for x in contents if ':Zone.Identifier' not in x]
        
        print()
        print('#'*60)
        print('Subject', subject)
        print(folder_path)
        print(contents)

        
        for algorithm in list_algorithm:
            print(algorithm)
            
            directory_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/image_input{algorithm}_{scale_percent}_{max_frames}'
            folder_save_path = os.path.join(directory_save_path, subject)
            
            # Create the output folder if it doesn't exist
            if not os.path.exists(folder_save_path):
                os.makedirs(folder_save_path)

            for content in contents:

                # Input
                video_path = os.path.join(folder_path, content)  # Replace with the actual path to the video file
                # Output
                if algorithm in ['']:
                    folder_save_path_video = os.path.join(folder_save_path, content.replace('.mp4', ''))
                elif algorithm in ['_colmap', '_mve', '_vggsfm']:
                    folder_save_path_video = os.path.join(folder_save_path, content.replace('.mp4', ''), 'images')
                
                # Extract frames from the video
                # Set max frames
                duration, frame_count, fps = get_video_duration(video_path)
                saved_frame_count = extract_frames(video_path, folder_save_path_video, fps, scale_percent, max_frames)

                print(f'{content.replace('.mp4', '')} - Scale: {scale_percent} - Max Frame: {max_frames} - Duration: {round(duration, 3)}s - No Frames: {saved_frame_count}')
                # print(f'Duration: {duration}s - Max Frame Set: {max_frames}')
                
                results = [folder, 
                        algorithm, 
                        scale_percent,
                        max_frames,
                        content.replace('.mp4', ''), 
                        saved_frame_count,
                        duration, frame_count, fps]
                df_results.append(results)

columns = ['Subject', 'Algorithm', 'Scale', 'Max_Frames', 'Video', 'Frames_Saved', 'Duration', 'Frame_Count', 'FPS']

df_results = pd.DataFrame(df_results, columns=columns)
directory_save_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE_Frames_input/'
df_results.to_csv(os.path.join(directory_save_path, 'data_summary.csv'), index=False)



############################################################
Subject 1
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/1. Subject 1
['1D0L.mp4', '1D0R.mp4']

1D0L - Scale: 100 - Max Frame: 5 - Duration: 10.0s - No Frames: 50
1D0R - Scale: 100 - Max Frame: 5 - Duration: 6.8s - No Frames: 30

############################################################
Subject 10
/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/10. Subject 10
['10D2L.mp4', '10D1R.mp4', '10D1L.mp4', '10D0R.mp4', '10D2R.mp4', '10D0L.mp4']

10D2L - Scale: 100 - Max Frame: 5 - Duration: 14.867s - No Frames: 70
10D1R - Scale: 100 - Max Frame: 5 - Duration: 10.567s - No Frames: 50
10D1L - Scale: 100 - Max Frame: 5 - Duration: 11.067s - No Frames: 55
10D0R - Scale: 100 - Max Frame: 5 - Duration: 12.667s - No Frames: 60
10D2R - Scale: 100 - Max Frame: 5 - Duration: 14.167s - No Frames: 70
10D0L - Scale: 100 - Max Frame: 5 - Duration: 10.9s - No Frames: 50

############################################################
Subject 11
/home